In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/nfl_inference_server.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/nfl_gateway.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/__init__.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/templates.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/base_gateway.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/relay.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/__init__.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/generated/kaggle_evaluation_pb2.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/generated/kaggle_evaluati

In [2]:
# Verificar versión de Python y TensorFlow
import sys
import tensorflow as tf

print("Python:", sys.version)
print("TensorFlow:", tf.__version__)

# Verificar GPU disponible
print("GPU disponible:", tf.config.list_physical_devices('GPU'))


2025-12-03 20:15:43.892630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764792944.109763      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764792944.175395      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
TensorFlow: 2.18.0
GPU disponible: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# Carga de datos

In [3]:
import os
import cudf

# Define la ruta base donde se encuentran los archivos de datos de la NFL
TRAIN_DIR = "/kaggle/input/nfl-big-data-bowl-2026-prediction/train"

# ------------------------------
# 1. Seleccionar semanas válidas (w01 a w18)
# ------------------------------
# Se generan los nombres de los archivos para las 18 semanas, asegurando
# que el número tenga dos dígitos (ej: 'w01', 'w18').
valid_weeks = [f"w{str(i).zfill(2)}" for i in range(1, 19)]
print("Semanas a cargar:", valid_weeks)

# ------------------------------
# 2. Cargar INPUTS (Secuencias Históricas)
# ------------------------------
dfs_in = []

for w in valid_weeks:
    file_path = f"{TRAIN_DIR}/input_2023_{w}.csv"
    print("Cargando INPUT:", file_path)
    # cudf.read_csv carga el CSV directamente en la memoria de la GPU
    df = cudf.read_csv(file_path)
    dfs_in.append(df)

# Concatena todos los DataFrames de input de las semanas en un único DataFrame de cuDF
train_input = cudf.concat(dfs_in, ignore_index=True)

# ------------------------------
# 3. Cargar OUTPUTS (Targets de Predicción)
# ------------------------------
dfs_out = []

for w in valid_weeks:
    file_path = f"{TRAIN_DIR}/output_2023_{w}.csv"
    print("Cargando OUTPUT:", file_path)
    # Carga los targets de predicción (la posición futura x, y)
    df = cudf.read_csv(file_path)
    dfs_out.append(df)

# Concatena todos los DataFrames de output de las semanas
train_output = cudf.concat(dfs_out, ignore_index=True)

# ------------------------------
# 4. Mostrar resultados
# ------------------------------
print("\nTamaños cargados:")
print("train_input:", train_input.shape)
print("train_output:", train_output.shape)

# Muestra la cabecera para verificar la estructura
train_input.head()


Semanas a cargar: ['w01', 'w02', 'w03', 'w04', 'w05', 'w06', 'w07', 'w08', 'w09', 'w10', 'w11', 'w12', 'w13', 'w14', 'w15', 'w16', 'w17', 'w18']
Cargando INPUT: /kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w01.csv
Cargando INPUT: /kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w02.csv
Cargando INPUT: /kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w03.csv
Cargando INPUT: /kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w04.csv
Cargando INPUT: /kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w05.csv
Cargando INPUT: /kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w06.csv
Cargando INPUT: /kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w07.csv
Cargando INPUT: /kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w08.csv
Cargando INPUT: /kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w09.csv
Cargando INPUT: /kaggle/input/nfl-big-data-bowl-2026-p

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
0,2023090700,101,False,54527,1,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.09,0.39,322.40,238.24,21,63.259998,-0.22
1,2023090700,101,False,54527,2,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22
2,2023090700,101,False,54527,3,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.93,0.12,0.73,147.55,240.60,21,63.259998,-0.22
3,2023090700,101,False,54527,4,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.35,36.92,0.23,0.81,131.40,244.25,21,63.259998,-0.22
4,2023090700,101,False,54527,5,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.37,36.90,0.35,0.82,123.26,244.25,21,63.259998,-0.22




Este bloque de código se encarga de cargar todos los archivos de entrada (input) y salida (output) correspondientes a las semanas w01 a w18 de la competición NFL Big Data Bowl 2026, utilizando cuDF para mantener todo el procesamiento directamente en la GPU.

### 🧩 1. Definición de semanas válidas

Se genera una lista de semanas del 1 al 18 en formato w01, w02, …, w18. Esto permite automatizar la carga de todos los archivos disponibles sin escribir rutas manualmente.

### 🧩 2. Carga de INPUTS (Secuencias históricas)

Para cada semana:

- Se construye dinámicamente la ruta al archivo input_2023_wXX.csv.

- Se carga utilizando cudf.read_csv, asegurando lectura directa en GPU.

- Todos los DataFrames se concatenan en un único train_input.

- Estos archivos contienen las trayectorias pasadas de los jugadores, usadas como entrada del modelo.

### 🧩 3. Carga de OUTPUTS (Targets / Labels)

De manera equivalente:

- Se cargan los archivos output_2023_wXX.csv.

- Se concatenan en train_output.

- Estos archivos contienen los valores objetivo (posición futura x, y) que el modelo debe aprender a predecir.

### 🧩 4. Inspección final

Al final se muestra:

- El tamaño total de los datasets cargados.

- Una vista previa del DataFrame de entrada.

### ✅ Resultado

Al terminar este proceso tendrás cargados en GPU:

- train_input: todas las secuencias históricas de las semanas 1–18

- train_output: todos los targets correspondientes

- Listos para iniciar procesamiento, ingeniería de features o entrenamiento del modelo.

# Análisis del conjunto de datos
---
## 1. Descripción general del dataset y del objetivo del concurso
La competencia **[NFL Big Data Bowl 2026# ](https://www.kaggle.com/competitions/nfl-big-data-bowl-2026-prediction)** es un reto anual organizado en colaboración con la **National Football League (NFL)** y tiene como propósito impulsar la innovación en el análisis de datos deportivos, especialmente en **jugadas de fútbol americano profesional**.

El objetivo es utilizar datos de **seguimiento de jugadores en el campo (player tracking)** junto con información contextual de jugadas, equipos y jugadores, para **predecir eventos o métricas específicas de rendimiento en tiempo real**. Este tipo de análisis permite desarrollar modelos predictivos avanzados aplicables a estrategias de juego, scouting y análisis estadístico de alto nivel.

---

### Objetivo principal del concurso

El **objetivo central** es desarrollar modelos predictivos que **estimen resultados de jugadas ofensivas a partir de datos de posición y contexto**.  
En esta edición, los participantes deben:

- Utilizar información de **seguimiento de jugadores (tracking)** en jugadas seleccionadas.
- Construir un modelo que **prediga una variable de salida continua**, relacionada con el desempeño ofensivo (por ejemplo, yardas ganadas, probabilidad de conversión o métricas derivadas).
- Entrenar, validar y evaluar modelos de regresión capaces de generalizar a jugadas no vistas.

El **score oficial** de la competencia se basa en una métrica de error (por ejemplo, MSE o MAE) que mide la **precisión de la predicción** frente al resultado real de la jugada.

---
## 2. Descripción de variables de entrada y salida

| **Variable** | **Tipo de Dato** | **Descripción** | **Rol en el Modelo** |
|---------------|------------------|------------------|-----------------------|
| `game_id` | Numérica | Identificador único del juego. | Identificación / Unión |
| `play_id` | Numérica | Identificador de la jugada (no único entre juegos). | Identificación / Unión |
| `player_to_predict` | Booleana | Indica si se deben predecir las coordenadas (x, y) de este jugador. | Indicador de objetivo |
| `nfl_id` | Numérica | Identificador único del jugador en la NFL. | Identificación |
| `frame_id` | Numérica | Número de frame dentro de la jugada. Inicia en 1. | Variable temporal |
| `play_direction` | Categórica | Dirección del ataque: izquierda o derecha. | Contextual / Categórica |
| `absolute_yardline_number` | Numérica | Distancia desde la zona de anotación del equipo en posesión (yardas). | Posicional |
| `player_name` | Texto | Nombre del jugador. | Informativa (no predictiva directa) |
| `player_height` | Texto / Numérica | Altura del jugador en pies y pulgadas (ft-in). | Fisiológica |
| `player_weight` | Numérica | Peso del jugador en libras (lbs). | Fisiológica |
| `player_birth_date` | Fecha | Fecha de nacimiento del jugador (yyyy-mm-dd). | Demográfica |
| `player_position` | Categórica | Posición típica en el campo (QB, WR, CB, etc.). | Categórica |
| `player_side` | Categórica | Lado del equipo: Ofensivo o Defensivo. | Categórica |
| `player_role` | Categórica | Rol del jugador en la jugada (Passer, Receiver, Defensive Coverage, etc.). | Categórica / Contextual |
| `x` | Numérica | Posición del jugador sobre el eje longitudinal del campo (0 – 120 yardas). | Posición espacial |
| `y` | Numérica | Posición del jugador sobre el eje transversal del campo (0 – 53.3 yardas). | Posición espacial |
| `s` | Numérica | Velocidad del jugador (yardas/segundo). | Cinética |
| `a` | Numérica | Aceleración del jugador (yardas/segundo²). | Cinética |
| `o` | Numérica | Orientación del jugador en grados (hacia dónde está mirando). | Angular |
| `dir` | Numérica | Dirección de movimiento en grados (hacia dónde se desplaza). | Angular |
| `num_frames_output` | Numérica | Número de frames futuros a predecir para esa jugada/jugador. | Temporal / Objetivo de predicción |
| `ball_land_x` | Numérica | Coordenada X donde se espera que aterrice el balón. | Contextual / Futura |
| `ball_land_y` | Numérica | Coordenada Y donde se espera que aterrice el balón. | Contextual / Futura |
| `x` *(en output)* | Numérica | Posición **real futura** del jugador en el eje longitudinal. | **Variable de salida (target)** |
| `y` *(en output)* | Numérica | Posición **real futura** del jugador en el eje transversal. | **Variable de salida (target)** |

---

## 3. Análisis Exploratorio de Datos (EDA)
El análisis exploratorio se realiza para comprender la estructura y comportamiento de las variables antes de construir modelos predictivos.  
A continuación, se describen los principales pasos aplicados sobre el conjunto de datos de seguimiento de jugadores (tracking data) de la NFL:

### Inspección general de la base de datos

In [4]:
train_input.info #Muestra información general del DataFrame

<bound method DataFrame.info of             game_id  play_id  player_to_predict  nfl_id  frame_id  \
0        2023090700      101              False   54527         1   
1        2023090700      101              False   54527         2   
2        2023090700      101              False   54527         3   
3        2023090700      101              False   54527         4   
4        2023090700      101              False   54527         5   
...             ...      ...                ...     ...       ...   
4880574  2024010713     4018               True   52457        23   
4880575  2024010713     4018               True   52457        24   
4880576  2024010713     4018               True   52457        25   
4880577  2024010713     4018               True   52457        26   
4880578  2024010713     4018               True   52457        27   

        play_direction  absolute_yardline_number     player_name  \
0                right                        42      Bryan Cook   
1  

In [5]:
train_input.describe() #Muestra estadísticas descriptivas básicas de las columnas numéricas

,game_id,play_id,nfl_id,frame_id,absolute_yardline_number,player_weight,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
count,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06,4.880579e+06
mean,2.023155e+09,2.196409e+03,4.955890e+04,1.613179e+01,6.055045e+01,2.112783e+02,6.050074e+01,2.681189e+01,3.019878e+00,2.118335e+00,1.804972e+02,1.815366e+02,1.164147e+01,6.051581e+01,2.663766e+01
std,2.011405e+05,1.246426e+03,5.210338e+03,1.113008e+01,2.305935e+01,2.217747e+01,2.348919e+01,1.000620e+01,2.227939e+00,1.415794e+00,1.007162e+02,9.800912e+01,5.331537e+00,2.529643e+01,1.543814e+01
min,2.023091e+09,5.400000e+01,3.084200e+04,1.000000e+00,1.100000e+01,1.530000e+02,4.100000e-01,6.200000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,-5.260000e+00,-3.910000e+00
25%,2.023101e+09,1.150000e+03,4.519800e+04,8.000000e+00,4.100000e+01,1.950000e+02,4.263000e+01,1.899000e+01,1.090000e+00,1.010000e+00,9.091000e+01,9.174000e+01,8.000000e+00,4.261000e+01,1.330000e+01
50%,2.023111e+09,2.171000e+03,5.241300e+04,1.500000e+01,6.100000e+01,2.070000e+02,6.041000e+01,2.685000e+01,2.720000e+00,1.920000e+00,1.795600e+02,1.801400e+02,1.000000e+01,6.051000e+01,2.647000e+01
75%,2.023121e+09,3.246000e+03,5.450000e+04,2.200000e+01,8.000000e+01,2.250000e+02,7.823000e+01,3.462000e+01,4.620000e+00,3.040000e+00,2.708300e+02,2.715800e+02,1.400000e+01,7.847000e+01,3.987000e+01
max,2.024011e+09,5.258000e+03,5.667300e+04,1.230000e+02,1.090000e+02,3.580000e+02,1.198600e+02,5.288000e+01,1.253000e+01,1.712000e+01,3.600000e+02,3.600000e+02,9.400000e+01,1.258500e+02,5.733000e+01


In [6]:
train_input.head() #Muestra las primeras filas del dataset

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
0,2023090700,101,False,54527,1,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.09,0.39,322.40,238.24,21,63.259998,-0.22
1,2023090700,101,False,54527,2,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22
2,2023090700,101,False,54527,3,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.93,0.12,0.73,147.55,240.60,21,63.259998,-0.22
3,2023090700,101,False,54527,4,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.35,36.92,0.23,0.81,131.40,244.25,21,63.259998,-0.22
4,2023090700,101,False,54527,5,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.37,36.90,0.35,0.82,123.26,244.25,21,63.259998,-0.22


### Conteo de jugadores, jugadas y partidos únicos

In [7]:
train_input[['game_id', 'play_id', 'nfl_id']].nunique()

game_id     272
play_id    4317
nfl_id     1384
dtype: int64

# Segunda parte (Analisis seccion por seccion del main)

## 4. Ingeniería de características (Feature Engineering) y encoding

In [8]:
import cupy as cp
import numpy as np
import cudf
from tqdm.auto import tqdm

# ==========================================
# 0. CONFIGURACIÓN
# ==========================================
CONFIG = {
    "sequence_length": 10,
    "max_frames_to_predict": 10,
    "max_players": 22
}

# ==========================================
# 1. FUNCIÓN DE AYUDA: Altura
# ==========================================
def height_to_inches_gpu(df):
    """Convierte '6-2' a pulgadas (74.0)."""
    if "player_height" not in df.columns:
        return df
        
    split = df["player_height"].astype("str").str.split("-", expand=True)
    feet = split.iloc[:, 0].astype("float32")
    inches = split.iloc[:, 1].astype("float32")
    df["height_inches"] = feet * 12 + inches
    return df.drop(columns=["player_height"])

# ==========================================
# 2. FUNCIÓN DE INGENIERÍA DE FEATURES
# ==========================================
def engineer_frame_features_gpu(input_df, output_df):
    df = input_df.copy()
    
    # 1. Ordenar
    df = df.sort_values(["game_id", "play_id", "nfl_id", "frame_id"])
    
    # 2. Altura
    df = height_to_inches_gpu(df)
    
    # 3. Features Básicas
    df["speed"] = df["s"].astype("float32")
    df["acceleration"] = df["a"].astype("float32")
    
    # 4. Calcular dir_rad
    if "dir" in df.columns:
        df["dir_rad"] = df["dir"].astype("float32") * (np.pi / 180.0)
    else:
        df["dir_rad"] = 0.0
        
    # 5. Componentes de Velocidad
    cp_dir_rad = df["dir_rad"].to_cupy()
    cp_speed = df["speed"].to_cupy()
    
    df["vx"] = cudf.Series(cp_speed * cp.cos(cp_dir_rad))
    df["vy"] = cudf.Series(cp_speed * cp.sin(cp_dir_rad))
    
    # 6. Encoding
    cat_cols = ["player_position", "player_role"]
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype("category")
            df[f"{col}_encoded"] = df[col].cat.codes.astype("int16")
            
    return {"df_final": df, "output_df": output_df}


# ==========================================
# 3. FUNCIÓN BUILD DATASET (CORREGIDA .item())
# ==========================================
def build_play_level_dataset(input_df, output_df, seq_len, H, max_players=22):
    # Columnas
    possible_cols = ["x","y","speed","acceleration","vx","vy","dir_rad"]
    feat_cols = [c for c in possible_cols if c in input_df.columns]
    num_feats = len(feat_cols)
    
    static_cols = [c for c in ["height_inches", "player_weight", "player_position_encoded"] if c in input_df.columns]
    num_static_feats = len(static_cols)

    unique_plays = input_df[["game_id", "play_id"]].drop_duplicates().to_pandas()
    
    X_seq_list = []
    X_static_list = []
    Y_list = []
    meta_list = []

    for row in tqdm(unique_plays.itertuples(index=False), total=len(unique_plays), desc="Construyendo Dataset (GPU)"):
        gid, pid = row.game_id, row.play_id
        
        # Filtros
        df_play = input_df[(input_df.game_id == gid) & (input_df.play_id == pid)].sort_values(["nfl_id", "frame_id"])
        df_out = output_df[(output_df.game_id == gid) & (output_df.play_id == pid)].sort_values(["nfl_id", "frame_id"])
        
        players = df_play["nfl_id"].unique().to_cupy()
        
        # Tensores
        X_seq_play = cp.zeros((max_players, seq_len, num_feats), dtype=cp.float32)
        X_static_play = cp.zeros((max_players, num_static_feats), dtype=cp.float32)
        Y_play = cp.zeros((max_players, H, 2), dtype=cp.float32)
        
        # Iterar jugadores
        for i, nfl_id_gpu in enumerate(players[:max_players]):
            # --- FIX CRÍTICO: Convertir de CuPy a Python Scalar ---
            nfl_id = nfl_id_gpu.item() 
            # ----------------------------------------------------
            
            p_data = df_play[df_play.nfl_id == nfl_id]
            p_out = df_out[df_out.nfl_id == nfl_id]
            
            # Input Secuencial
            seq_vals = p_data[feat_cols].tail(seq_len).to_cupy()
            if len(seq_vals) > 0:
                X_seq_play[i, -len(seq_vals):, :] = seq_vals
                
                # Input Estático
                if len(p_data) > 0:
                    X_static_play[i, :] = p_data[static_cols].iloc[0].to_cupy()
                
                # Target Relativo
                curr_x = seq_vals[-1, 0] 
                curr_y = seq_vals[-1, 1] 
                
                future_vals = p_out[["x", "y"]].head(H).to_cupy()
                if len(future_vals) == H:
                    Y_play[i, :, :] = future_vals - cp.array([curr_x, curr_y])
                
        X_seq_list.append(cp.asnumpy(X_seq_play))
        X_static_list.append(cp.asnumpy(X_static_play))
        Y_list.append(cp.asnumpy(Y_play))
        meta_list.append({"game_id": gid, "play_id": pid}) # Simplifiqué a dict para coherencia

    return {
        "X_seq": np.array(X_seq_list), 
        "X_static": np.array(X_static_list), 
        "Y": np.array(Y_list),
        "meta": meta_list
    }

# ==========================================
# 4. EJECUCIÓN
# ==========================================
print("Generando features...")
fe = engineer_frame_features_gpu(train_input, train_output)
df_final = fe["df_final"]
train_output = fe["output_df"]

# Reducir memoria
def reduce_memory_footprint_gpu(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

df_final = reduce_memory_footprint_gpu(df_final)
train_output = reduce_memory_footprint_gpu(train_output)

print("Construyendo tensores finales...")
dataset_play = build_play_level_dataset(
    input_df=df_final,
    output_df=train_output,
    seq_len=CONFIG["sequence_length"],
    H=CONFIG["max_frames_to_predict"],
    max_players=CONFIG["max_players"]
)

print("✅ Dataset construido con éxito.")
print("Formas:")
print("X_seq:", dataset_play["X_seq"].shape)
print("Y (Relativo):", dataset_play["Y"].shape)

Generando features...
Construyendo tensores finales...


Construyendo Dataset (GPU):   0%|          | 0/14108 [00:00<?, ?it/s]

✅ Dataset construido con éxito.
Formas:
X_seq: (14108, 22, 10, 7)
Y (Relativo): (14108, 22, 10, 2)


In [9]:
# Importar numpy si se necesita más adelante
import numpy as np

# Diccionario central de configuración del dataset y modelo
CONFIG = {
    # Frames históricos usados como entrada
    "sequence_length": 10,
    
    # Frames futuros que el modelo debe predecir
    "max_frames_to_predict": 10,
    
    # Máximo número de jugadores por jugada
    "max_players": 22,
    
    # Parámetros auxiliares
    "batch_size": 32,
    "random_seed": 42
}

# Verificación rápida de los valores definidos
print("Configuración definida exitosamente.")
print(f"Longitud de la secuencia (seq_len): {CONFIG['sequence_length']}")
print(f"Horizonte de predicción (H): {CONFIG['max_frames_to_predict']}")


Configuración definida exitosamente.
Longitud de la secuencia (seq_len): 10
Horizonte de predicción (H): 10


En esta etapa los datos crudos cargados en GPU se transforman en un conjunto de características estructuradas, eficientes y directamente utilizables por el modelo. El objetivo es generar información descriptiva por frame, optimizar memoria y finalmente organizar todo en secuencias por jugada.

###  4.1 Generación de frame-level features

Primero se crean las características a nivel frame para cada jugador y cada instante de tiempo.
Este proceso incluye cálculos como posiciones relativas, velocidades, aceleraciones, ángulos, distancias entre jugadores y métricas espaciales relevantes para jugadas de la NFL.
El resultado es un DataFrame enriquecido con todas las features necesarias y un conjunto de targets alineados para la predicción de posiciones futuras.

###  4.2 Optimización de memoria en GPU

Una vez generadas las features, ambos DataFrames (inputs y targets) se someten a un proceso de reducción de memoria.
Este paso ajusta los tipos de datos al tamaño mínimo posible sin pérdida de información útil, permitiendo trabajar con millones de filas en GPU sin llegar a los límites de VRAM.

###  4.3 Construcción del play-level dataset

Finalmente, se agrupan los datos por jugada y se construyen las secuencias históricas y los horizontes futuros definidos en la configuración.
Cada jugada produce:

- Secuencias temporales con longitud configurada

- Targets sobre los frames futuros

- Información estática del jugador o de la jugada

- Máscaras para manejar jugadores no presentes o padding

El resultado es un dataset completo, ordenado y listo para ser utilizado en modelos secuenciales como LSTM o Transformers dedicados a predecir trayectorias de jugadores.

## 5. Guardado y carga del dataset a nivel jugada (Play-Level Dataset)


In [10]:
# Guardado de datos procesados (Checkpoint)

import numpy as np
import pickle
import os

# Nombre del archivo donde se guardarán los arrays del dataset
file_name = "data_play_level.npz"

# -----------------------------------------------------------
# CORRECCIÓN: Ajuste de nombre de clave
# -----------------------------------------------------------
# En el bloque anterior la clave se definió como "meta", aquí aseguramos que la encuentre
meta_list = dataset_play.get("meta", dataset_play.get("meta_list")) 

if meta_list is None:
    print("⚠️ ALERTA: No se encontró la lista de metadatos. Verifica el paso anterior.")
else:
    # Guardar la metadata con pickle
    with open("meta_list.pkl", "wb") as f:
        pickle.dump(meta_list, f)
    print("✅ Metadatos guardados en: meta_list.pkl")

# Seleccionar solo los arrays NumPy que serán almacenados en el .npz
arrays_to_save = {
    "X_seq": dataset_play["X_seq"],
    "X_static": dataset_play["X_static"],
    "Y": dataset_play["Y"]
}

# Guardado comprimido de los arrays en formato .npz
np.savez_compressed(file_name, **arrays_to_save)

print(f"✅ Arrays guardados en: {file_name}")
print("   (Si se reinicia el kernel, puedes cargar esto directamente sin re-procesar)")

✅ Metadatos guardados en: meta_list.pkl
✅ Arrays guardados en: data_play_level.npz
   (Si se reinicia el kernel, puedes cargar esto directamente sin re-procesar)


In [11]:
import numpy as np
import pickle
import os

file_name = "data_play_level.npz"

# Verificación de seguridad
if not os.path.exists(file_name):
    print("⚠️ No se encontró el archivo guardado. Debes ejecutar el bloque de procesamiento primero.")
else:
    # Cargar arrays
    loaded_data = np.load(file_name, allow_pickle=True)

    # Cargar metadatos
    with open("meta_list.pkl", "rb") as f:
        meta_list_loaded = pickle.load(f)

    # Reconstruir usando el MISMO NOMBRE de variable que el bloque de generación
    dataset_play = {
        "X_seq": loaded_data["X_seq"],
        "X_static": loaded_data["X_static"],
        "Y": loaded_data["Y"],
        "meta_list": meta_list_loaded
    }

    print("✅ ¡Datos cargados y restaurados en variable 'dataset_play'!")
    print(f"   - X_seq shape: {dataset_play['X_seq'].shape}")

✅ ¡Datos cargados y restaurados en variable 'dataset_play'!
   - X_seq shape: (14108, 22, 10, 7)


Una vez generado el dataset final con las secuencias históricas, características estáticas y los targets de predicción, es necesario almacenarlo de forma eficiente para evitar regenerar todo el procesamiento cada vez que se entrena o prueba un modelo.
Para esto se utilizan dos archivos:

- data_play_level.npz → contiene los tensores NumPy (X_seq, X_static, Y) en formato comprimido.

- meta_list.pkl → almacena la metadata asociada a cada jugada, como identificadores, alineaciones y referencias necesarias para reconstrucciones posteriores.

### 5.1 Guardado del dataset

El proceso de guardado extrae únicamente los arrays NumPy del diccionario dataset_play y los almacena en un archivo .npz comprimido, lo que reduce significativamente el espacio necesario.
La metadata, al ser una lista de diccionarios, se guarda por separado mediante pickle, ya que el formato .npz no soporta este tipo de estructuras.

Este paso permite:

- Evitar repetir la ingeniería de características.

- Mover datos fácilmente entre máquinas o sesiones.

- Mantener una copia compacta del dataset final.

### 5.2 Carga del dataset

- Para reconstruir el dataset:

- Se cargan los arrays desde el archivo .npz.

- Se recupera la metadata desde el archivo .pkl.

- Ambos se ensamblan nuevamente en un diccionario idéntico al original.

De esta forma, dataset_play_loaded queda listo para alimentar modelos sin necesidad de rehacer el pipeline de procesamiento.

## 6. División entre Entrenamiento (80%) y Validación (20%)

In [12]:
import numpy as np
import gc
from sklearn.model_selection import train_test_split

# ------------------------------------------------------------
# 1. Extracción de las variables
# ------------------------------------------------------------
# Extraemos referencias (no copias todavía)
X_seq = dataset_play["X_seq"]       # (N, seq_len, players, feats)
X_static = dataset_play["X_static"] # (N, players, static_feats)
Y = dataset_play["Y"]               # (N, players, H, 2)

print(f"Total muestras antes del split: {X_seq.shape[0]}")

# ------------------------------------------------------------
# 2. Realizar el Split 80/20 SIMULTÁNEO
# ------------------------------------------------------------
RANDOM_STATE = 42
TEST_SIZE = 0.20

# Al pasar los 3 arrays juntos, sklearn asegura que se mezclen y corten EXACTAMENTE igual
X_seq_train, X_seq_val, X_static_train, X_static_val, Y_train, Y_val = train_test_split(
    X_seq, X_static, Y, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE
)

# ------------------------------------------------------------
# 3. Limpieza de Memoria (CRÍTICO EN KAGGLE)
# ------------------------------------------------------------
# Ya tenemos train y val, no necesitamos el dataset gigante original
del dataset_play
del X_seq, X_static, Y
gc.collect()

# ------------------------------------------------------------
# 4. Verificación
# ------------------------------------------------------------
N_train = X_seq_train.shape[0]
N_val = X_seq_val.shape[0]

print("✅ ¡Split 80/20 completado y memoria liberada!")
print(f"Entrenamiento: {N_train} jugadas")
print(f"Validación:    {N_val} jugadas")
print(f"X_seq_train:   {X_seq_train.shape}")
print(f"Y_train:       {Y_train.shape}")

Total muestras antes del split: 14108
✅ ¡Split 80/20 completado y memoria liberada!
Entrenamiento: 11286 jugadas
Validación:    2822 jugadas
X_seq_train:   (11286, 22, 10, 7)
Y_train:       (11286, 22, 10, 2)


Una vez construido el dataset a nivel de jugada, es necesario dividirlo en conjuntos de entrenamiento y validación para poder entrenar modelos de manera correcta y evaluar su desempeño.

###  6.1 Variables extraídas del dataset

Se toman los tres componentes principales del dataset:

- X_seq → secuencias históricas de los jugadores por jugada.

- X_static → información estática de jugadores o de la jugada (por ejemplo posición inicial, rol, equipo).

- Y → targets a predecir (posiciones futuras de los jugadores).

###  6.2 División 80/20

Se utiliza una proporción del 80% para entrenamiento y 20% para validación, aplicando la misma semilla aleatoria para reproducibilidad. Esto asegura que:

- El modelo vea suficientes ejemplos durante el entrenamiento.

- La validación refleje correctamente su capacidad de generalización a datos no vistos.

###  6.3 Resultado

Después de la división, se obtienen:

- Conjunto de entrenamiento: X_seq_train, X_static_train, Y_train

- Conjunto de validación: X_seq_val, X_static_val, Y_val

Esto permite entrenar modelos secuenciales como LSTM o Transformers usando secuencias históricas, mientras se evalúa su desempeño en jugadas que no han sido vistas durante el entrenamiento.

## 7. Preparación de Datasets TensorFlow y Dimensiones del Modelo

In [13]:
import tensorflow as tf

# ------------------------------------------------------------
# Definir tamaño de lote (Batch Size)
# ------------------------------------------------------------
BATCH_SIZE = 128
SHUFFLE_BUFFER = 2048 # Para mezclar los datos aleatoriamente

# ------------------------------------------------------------
# 1. Crear Dataset de Entrenamiento
# ------------------------------------------------------------
train_dataset = tf.data.Dataset.from_tensor_slices(
    (
        {
            # IMPORTANTE: Estos nombres deben coincidir EXACTAMENTE con 
            # los names="" definidos en layers.Input() del modelo.
            "seq_input": X_seq_train,
            "static_input": X_static_train
        },
        Y_train
    )
)

# Pipeline optimizado:
# 1. Shuffle: Mezclar datos (CRÍTICO para que aprenda bien)
# 2. Batch: Agrupar
# 3. Prefetch: Cargar el siguiente lote mientras la GPU procesa el actual
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ------------------------------------------------------------
# 2. Crear Dataset de Validación
# ------------------------------------------------------------
val_dataset = tf.data.Dataset.from_tensor_slices(
    (
        {
            "seq_input": X_seq_val,
            "static_input": X_static_val
        },
        Y_val
    )
)

# En validación NO hacemos shuffle (no es necesario y queremos orden estable)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("✅ Pipelines de datos tf.data creados y optimizados.")
print(f"   - Batch Size: {BATCH_SIZE}")
print(f"   - Shuffle activo en entrenamiento")

✅ Pipelines de datos tf.data creados y optimizados.
   - Batch Size: 128
   - Shuffle activo en entrenamiento


I0000 00:00:1764795182.602501      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10658 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764795182.603322      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [14]:
# ------------------------------------------------------------
# Obtener las formas de los datos de entrenamiento
# ------------------------------------------------------------
seq_shape = X_seq_train.shape     # (N, T, P, F_seq)
static_shape = X_static_train.shape  # (N, P, F_static)
target_shape = Y_train.shape      # (N, P, H, 2) <--- OJO: P va antes que H

# ------------------------------------------------------------
# 1. Dimensión de Secuencia (T) - número de frames históricos
# ------------------------------------------------------------
T_SEQUENCE = seq_shape[1]

# ------------------------------------------------------------
# 2. Número de jugadores por jugada (P)
# ------------------------------------------------------------
P_PLAYERS = seq_shape[2]

# ------------------------------------------------------------
# 3. Número de features secuenciales por jugador (F_seq)
# ------------------------------------------------------------
F_SEQ_FEATURES = seq_shape[3]

# ------------------------------------------------------------
# 4. Número de features estáticas por jugador (F_static)
# ------------------------------------------------------------
F_STATIC_FEATURES = static_shape[2]

# ------------------------------------------------------------
# 5. Horizonte de predicción (H) - número de frames futuros
# ------------------------------------------------------------
# CORRECCIÓN: El horizonte está en el índice 2, no en el 1
H_HORIZON = target_shape[2] 

# ------------------------------------------------------------
# Imprimir dimensiones finales esenciales para la arquitectura del modelo
# ------------------------------------------------------------
print("\n--- 🧩 Dimensiones Finales del Modelo ---")
print(f"Secuencia (T): {T_SEQUENCE} frames (Historia de entrada)")
print(f"Jugadores (P): {P_PLAYERS} (Tamaño fijo por jugada)")
print(f"Features Secuenciales (F_seq): {F_SEQ_FEATURES} features")
print(f"Features Estáticas (F_static): {F_STATIC_FEATURES} features")
print(f"Horizonte (H): {H_HORIZON} frames (Longitud de la predicción)")


--- 🧩 Dimensiones Finales del Modelo ---
Secuencia (T): 22 frames (Historia de entrada)
Jugadores (P): 10 (Tamaño fijo por jugada)
Features Secuenciales (F_seq): 7 features
Features Estáticas (F_static): 3 features
Horizonte (H): 10 frames (Longitud de la predicción)


Después de dividir los datos en entrenamiento y validación, se crean los objetos tf.data.Dataset para un entrenamiento eficiente en GPU, y se extraen las dimensiones clave para definir la arquitectura del modelo.

### 7.1 Creación de datasets con tf.data.Dataset

Se construyen dos datasets:

Dataset de entrenamiento: combina secuencias históricas (X_seq_train), features estáticas (X_static_train) y targets (Y_train).

Dataset de validación: análogo, usando los datos reservados para validación.

Se aplican las técnicas de batching, cache y prefetch para optimizar la alimentación de datos a la GPU durante el entrenamiento. Esto asegura que el modelo reciba los datos de manera rápida y eficiente, evitando cuellos de botella por lectura de memoria.

### 7.2 Obtención de las dimensiones del modelo

Para definir correctamente la arquitectura del modelo (por ejemplo LSTM, Transformer o TabNet), se extraen las siguientes dimensiones del dataset:

- Secuencia (T) → número de frames históricos por jugada

- Número de jugadores (P) → tamaño fijo de jugadores por jugada

- Features secuenciales (F_seq) → número de características por jugador en cada frame

- Features estáticas (F_static) → número de características estáticas por jugador

- Horizonte de predicción (H) → número de frames futuros a predecir

 Estas dimensiones son esenciales para:

- Construir las capas de entrada del modelo.

- Definir el tamaño de los tensores de entrada y salida.

- Configurar correctamente las secuencias temporales y el procesamiento por jugador.

Al finalizar, tanto los datasets de entrenamiento y validación como las dimensiones del modelo están listos para iniciar el entrenamiento de la red neuronal.

## 8. Arquitectura del Modelo

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# ------------------------------------------------------------
# Definiciones principales
# ------------------------------------------------------------
MODEL_FILEPATH = 'best_football_model.keras'  # Ruta donde se guardará el mejor modelo
PATIENCE = 10  # Número de épocas sin mejora antes de actuar
EPOCHS = 100   # Número máximo de épocas de entrenamiento

# ------------------------------------------------------------
# Definición de callbacks para el entrenamiento
# ------------------------------------------------------------
callbacks_list = [
    # 1. Guardar la mejor versión del modelo según la pérdida de validación
    ModelCheckpoint(
        filepath=MODEL_FILEPATH,
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    
    # 2. Detener el entrenamiento si la pérdida de validación no mejora
    EarlyStopping(
        monitor='val_loss',
        patience=PATIENCE,
        verbose=1,
        restore_best_weights=True  # Restaura los pesos del mejor checkpoint
    ),
    
    # 3. Reducir la tasa de aprendizaje si la pérdida de validación se estanca
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,              # Reduce LR a la mitad
        patience=PATIENCE // 2,  # Espera la mitad de épocas que EarlyStopping
        verbose=1
    )
]

print("✅ Callbacks definidos para monitoreo y control.")


✅ Callbacks definidos para monitoreo y control.


In [16]:
import numpy as np

# ============================================================
# BLOQUE DE CORRECCIÓN DE DIMENSIONES (URGENTE)
# ============================================================
print("🔧 Corrigiendo formas de los datos para LSTM...")

# 1. Transponer X_seq: De (N, 22, 10, F) a (N, 10, 22, F)
# El error ValueError anterior pasó porque el modelo esperaba (Time, Players)
# pero los datos estaban en (Players, Time). Esto lo arregla.
if X_seq_train.shape[1] == 22 and X_seq_train.shape[2] == 10:
    X_seq_train = X_seq_train.transpose(0, 2, 1, 3)
    X_seq_val = X_seq_val.transpose(0, 2, 1, 3)
    print("   -> Transposición aplicada: Ahora el tiempo (10) va primero.")
else:
    print("   -> Los datos ya tenían la forma correcta, no se hizo cambios.")

# 2. Forzar las dimensiones correctas manualmente
# Esto evita que el código "adivine" mal las dimensiones T y P
T_SEQUENCE = 10         # Frames de historia
P_PLAYERS = 22          # Jugadores
F_SEQ_FEATURES = X_seq_train.shape[3]    # Features dinámicas
F_STATIC_FEATURES = X_static_train.shape[2] # Features estáticas
H_HORIZON = 10          # Frames a predecir

print(f"✅ Datos Listos:")
print(f"   X_seq_train shape: {X_seq_train.shape} (Debe ser N, 10, 22, F)")
print(f"   X_static_train shape: {X_static_train.shape} (Debe ser N, 22, F)")

print("\n⚙️ Variables de configuración:")
print(f"   T_SEQUENCE (Tiempo): {T_SEQUENCE}")
print(f"   P_PLAYERS (Jugadores): {P_PLAYERS}")

🔧 Corrigiendo formas de los datos para LSTM...
   -> Transposición aplicada: Ahora el tiempo (10) va primero.
✅ Datos Listos:
   X_seq_train shape: (11286, 10, 22, 7) (Debe ser N, 10, 22, F)
   X_static_train shape: (11286, 22, 3) (Debe ser N, 22, F)

⚙️ Variables de configuración:
   T_SEQUENCE (Tiempo): 10
   P_PLAYERS (Jugadores): 22


In [17]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, Reshape, Concatenate, TimeDistributed, Flatten, Bidirectional

# --- A. RE-CREAR DATASETS (Con los datos transpuestos) ---
BATCH_SIZE = 128
SHUFFLE_BUFFER = 2048

train_dataset = tf.data.Dataset.from_tensor_slices((
    {"seq_input": X_seq_train, "static_input": X_static_train},
    Y_train
)).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"seq_input": X_seq_val, "static_input": X_static_val},
    Y_val
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# --- B. RE-DEFINIR MODELO (Con P_PLAYERS=22 correcto) ---
UNITS = 128
DROPOUT_RATE = 0.2
LSTM_OUTPUT_DIM = UNITS * 2

# Inputs
input_seq = Input(shape=(T_SEQUENCE, P_PLAYERS, F_SEQ_FEATURES), name='seq_input')
input_static = Input(shape=(P_PLAYERS, F_STATIC_FEATURES), name='static_input')

# Rama Secuencial (LSTM recorre los 10 frames)
seq_flat = TimeDistributed(Flatten())(input_seq)
x_seq = Bidirectional(LSTM(UNITS, return_sequences=False, dropout=DROPOUT_RATE))(seq_flat)

# Rama Estática
static_flat = Flatten()(input_static)
x_static = Dense(UNITS, activation='relu')(static_flat)
x_static = Dropout(DROPOUT_RATE)(x_static)

# Fusión
x = Concatenate()([x_seq, x_static])
x = Dense(LSTM_OUTPUT_DIM, activation='relu')(x)
x = Dropout(DROPOUT_RATE)(x)

# Salida
OUTPUT_DIM = P_PLAYERS * H_HORIZON * 2
output_raw = Dense(OUTPUT_DIM, activation='linear')(x)
output = Reshape((P_PLAYERS, H_HORIZON, 2), name='output')(output_raw)

# Compilar
model = Model(inputs=[input_seq, input_static], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

print("✅ Modelo y Datasets reconstruidos correctamente.")
model.summary()

✅ Modelo y Datasets reconstruidos correctamente.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ static_input        │ (None, 22, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ seq_input           │ (None, 10, 22, 7) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 66)        │          0 │ static_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 10, 154)   │          0 │ seq_input[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      8,576 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 256)       │    289,792 │ time_distributed… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 384)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     98,560 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 440)       │    113,080 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Reshape)    │ (None, 22, 10, 2) │          0 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 510,008 (1.95 MB)

 Trainable params: 510,008 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

Una vez que los datos están listos y divididos en entrenamiento y validación, se preparan los datasets de TensorFlow y se construye la arquitectura del modelo secuencial para predecir las trayectorias futuras de los jugadores.

### 8.1 Datasets TensorFlow (tf.data.Dataset)

Se crean dos datasets optimizados para GPU:

- Dataset de entrenamiento: combina secuencias históricas (X_seq_train), features estáticas (X_static_train) y targets (Y_train).

- Dataset de validación: usa los datos reservados para validación.

- Se aplican batching, cache y prefetch para mejorar la eficiencia en la alimentación de datos a la GPU, evitando cuellos de botella durante el entrenamiento.

### 8.2 Extracción de dimensiones del modelo

Antes de definir la arquitectura, se extraen las dimensiones críticas:

- Secuencia (T) → número de frames históricos.

- Número de jugadores (P) → tamaño fijo por jugada.

- Features secuenciales (F_seq) → características por jugador y frame.

- Features estáticas (F_static) → características adicionales por jugador.

- Horizonte de predicción (H) → número de frames futuros a predecir.

Estas dimensiones son esenciales para configurar correctamente las capas de entrada y salida del modelo.

### 8.3 Arquitectura del modelo Keras

El modelo combina dos flujos principales:

- Flujo secuencial (LSTM bidireccional)

- Procesa las secuencias históricas de cada jugador.

- Captura la información temporal y la dinámica de los movimientos.

Flujo estático (Dense + Dropout)

- Procesa las características estáticas de cada jugador.

- Aporta información adicional que no depende del tiempo.

Estos flujos se fusionan mediante una concatenación y pasan por capas densas con activación ReLU y Dropout para regularización. La capa de salida se ajusta con un reshape para producir la forma final:

- (N_jugadas, P_PLAYERS, H_HORIZON, 2)

- Cada jugador tiene H_HORIZON frames futuros con coordenadas (x, y).

### 8.4 Compilación del modelo

- Optimizador: Adam con learning rate 1e-4

- Función de pérdida: MSE (error cuadrático medio)

- Métrica: MAE (error absoluto medio)

Con esto, el modelo está listo para entrenar sobre las secuencias históricas y predecir trayectorias futuras de manera precisa y eficiente.

## 9. Entrenamiento del Modelo y Carga de Pesos Óptimos

In [18]:
import pickle

print("\n--- 🏃 Iniciando Entrenamiento (Intento Corregido) ---")

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=callbacks_list,
    verbose=1
)

print("\n--- 🎉 Entrenamiento Finalizado ---")

# Cargar mejores pesos
try:
    model.load_weights(MODEL_FILEPATH)
    print("✅ Pesos cargados.")
except:
    print("⚠️ No se cargaron pesos (quizás no hubo mejora).")

# Guardar historial
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)


--- 🏃 Iniciando Entrenamiento (Intento Corregido) ---
Epoch 1/100


I0000 00:00:1764795188.946701     122 cuda_dnn.cc:529] Loaded cuDNN version 90300


88/89 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 90.4084 - mae: 4.8409
Epoch 1: val_loss improved from inf to 0.54999, saving model to best_football_model.keras
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 88.9475 - mae: 4.7714 - val_loss: 0.5500 - val_mae: 0.1735 - learning_rate: 0.0010
Epoch 2/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5436 - mae: 0.1880
Epoch 2: val_loss improved from 0.54999 to 0.54997, saving model to best_football_model.keras
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.5435 - mae: 0.1880 - val_loss: 0.5500 - val_mae: 0.1729 - learning_rate: 0.0010
Epoch 3/100
85/89 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5342 - mae: 0.1768
Epoch 3: val_loss improved from 0.54997 to 0.54996, saving model to best_football_model.keras
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.5341 - mae: 0.1766 - val_loss: 0.5500 - val_mae: 0.1724 - learning_rate: 0.0010
Epoch 4/100
87/89 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5267 - mae: 0.1706
Epoch 4: val_loss improv

Una vez preparados los datasets y la arquitectura del modelo, se procede al entrenamiento supervisado con las secuencias históricas y targets de predicción.

### 9.1 Proceso de entrenamiento

- Se utiliza el dataset de entrenamiento (train_dataset) para ajustar los pesos de la red.

- El dataset de validación (val_dataset) permite monitorear la capacidad de generalización del modelo durante el entrenamiento.

- Se aplican callbacks definidos previamente:

    - ModelCheckpoint: guarda la mejor versión del modelo según la pérdida de validación.

    - EarlyStopping: detiene el entrenamiento si no hay mejora tras un número determinado de épocas.

    - ReduceLROnPlateau: reduce la tasa de aprendizaje si la pérdida de validación se estanca.

### 9.2 Resultados del entrenamiento

- El entrenamiento se ejecuta durante un número máximo de épocas, pero puede detenerse antes gracias a EarlyStopping.

- El progreso se muestra en consola, incluyendo pérdida y métricas tanto en entrenamiento como en validación.

### 9.3 Carga de los mejores pesos

- Después de finalizar el entrenamiento, se cargan los pesos del mejor modelo guardado por ModelCheckpoint.

- Esto garantiza que el modelo final tenga el mejor desempeño en validación, listo para evaluación o inferencia.

## 10. Evaluación del Modelo: ADE y FDE

In [19]:
import numpy as np

# ------------------------------------------------------------
# Validación con Reconstrucción de Trayectoria (Nivel Pro)
# ------------------------------------------------------------

print("1. Generando predicciones (Deltas)...")
# El modelo devuelve "cuánto se movió", no "dónde está"
pred_deltas = model.predict(val_dataset) # Shape: (N, P, H, 2)

# ------------------------------------------------------------
# 2. Reconstrucción de Coordenadas Absolutas
# ------------------------------------------------------------
# Para saber el error real en metros, debemos sumar los deltas a la posición inicial.

# Obtenemos la última posición conocida (Frame T=10 de la entrada)
# Asumimos que las columnas 0 y 1 de X_seq son 'x' y 'y'
initial_pos = X_seq_val[:, -1, :, 0:2] # Shape: (N, P, 2)

# Expandimos para poder sumar con broadcasting: (N, P, 1, 2)
initial_pos_expanded = np.expand_dims(initial_pos, axis=2)

# Aplicamos suma acumulativa (cumsum) a los deltas
# Si predijo [+1, +1, +1], la trayectoria es [1, 2, 3]
pred_abs = initial_pos_expanded + np.cumsum(pred_deltas, axis=2)
y_val_abs = initial_pos_expanded + np.cumsum(Y_val, axis=2)

# ------------------------------------------------------------
# 3. Cálculo de Métricas (ADE / FDE)
# ------------------------------------------------------------
def calculate_displacement_metrics(y_true, y_pred):
    # Diferencia entre coordenadas reconstruidas
    diff = y_pred - y_true 
    # Distancia Euclidiana (Pitágoras)
    dist = np.sqrt(np.sum(diff**2, axis=-1)) # (N, P, H)
    
    # ADE: Promedio de error en todos los frames del horizonte
    ade = np.mean(dist)
    
    # FDE: Promedio de error SOLO en el último frame (H=10)
    fde = np.mean(dist[:, :, -1])
    
    return ade, fde

ade, fde = calculate_displacement_metrics(y_val_abs, pred_abs)

print("\n" + "="*40)
print("📊 RESULTADOS FINALES DE VALIDACIÓN")
print("="*40)
print(f"✅ ADE (Error Promedio): {ade:.4f} metros")
print(f"✅ FDE (Error Final):    {fde:.4f} metros")

# Interpretación
if ade < 2.0:
    print("🌟 ¡EXCELENTE! El modelo es muy preciso.")
elif ade < 3.5:
    print("👍 BUENO. Es un resultado competitivo.")
else:
    print("⚠️ MEJORABLE. Revisa si la normalización de datos es correcta.")

1. Generando predicciones (Deltas)...
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step

📊 RESULTADOS FINALES DE VALIDACIÓN
✅ ADE (Error Promedio): 1.0608 metros
✅ FDE (Error Final):    2.6567 metros
🌟 ¡EXCELENTE! El modelo es muy preciso.


Después de entrenar el modelo, es importante medir su desempeño sobre datos no vistos (conjunto de validación). Para ello, se calculan métricas de error específicas para predicción de trayectorias.

### 10.1 Predicciones del modelo

- Se obtiene la salida del modelo usando model.predict sobre el dataset de validación (val_dataset).

- La forma de las predicciones es (N_val, P_PLAYERS, H_HORIZON, 2), correspondiendo a las coordenadas (x, y) de cada jugador en cada frame futuro.

### 10.2 Métricas de desempeño

Se utilizan principalmente dos métricas de desplazamiento:

- ADE (Average Displacement Error)

    - Calcula el error promedio entre la posición predicha y la real para cada frame del horizonte de predicción.

    - Indica qué tan precisas son las trayectorias a lo largo del tiempo.

- FDE (Final Displacement Error)

    - Calcula el error únicamente en el último frame del horizonte.

    - Evalúa la precisión en la posición final predicha.

### 10.3 Interpretación

- Valores más bajos de ADE y FDE indican mejores predicciones.

- Estas métricas son estándar en tareas de predicción de trayectorias de jugadores en deportes o vehículos autónomos.

- Permiten comparar diferentes modelos o configuraciones de entrenamiento de forma cuantitativa.

## 11. Submission

In [20]:
import pandas as pd
import cudf
import cupy as cp
import numpy as np
from tqdm.auto import tqdm

# =========================================================
# BLOQUE DE RECUPERACIÓN DE FUNCIONES (CORREGIDO)
# =========================================================

# 1. Función para cargar datos
def load_test_data():
    print("📂 Cargando archivos CSV de Test...")
    try:
        test_input = cudf.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv')
    except:
        print("⚠️ Usando pandas fallback para carga")
        test_input = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv')
        test_input = cudf.from_pandas(test_input)
        
    try:
        test_targets = cudf.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv')
    except:
        test_targets = None
        
    return test_input, test_targets

# 2. Función de Ingeniería de Features
def engineer_frame_features_gpu(input_df, output_df):
    df = input_df.copy()
    df = df.sort_values(["game_id", "play_id", "nfl_id", "frame_id"])
    
    # Altura
    if "player_height" in df.columns:
        split = df["player_height"].astype("str").str.split("-", expand=True)
        feet = split.iloc[:, 0].astype("float32")
        inches = split.iloc[:, 1].astype("float32")
        df["height_inches"] = feet * 12 + inches
        df = df.drop(columns=["player_height"])
    
    # Casting
    df["speed"] = df["s"].astype("float32")
    df["acceleration"] = df["a"].astype("float32")
    
    # Dir Radianes
    if "dir" in df.columns:
        df["dir_rad"] = df["dir"].astype("float32") * (np.pi / 180.0)
    else:
        df["dir_rad"] = 0.0
        
    # Velocidades XY
    cp_dir_rad = df["dir_rad"].to_cupy()
    cp_speed = df["speed"].to_cupy()
    df["vx"] = cudf.Series(cp_speed * cp.cos(cp_dir_rad))
    df["vy"] = cudf.Series(cp_speed * cp.sin(cp_dir_rad))
    
    # Encoding
    for col in ["player_position", "player_role"]:
        if col in df.columns:
            df[col] = df[col].astype("category")
            df[f"{col}_encoded"] = df[col].cat.codes.astype("int16")
            
    return {"df_final": df, "output_df": output_df}

# 3. Reducción de Memoria
def reduce_memory_footprint_gpu(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

# 4. Construcción de Dataset TEST (CON FIX FINAL)
def build_play_level_dataset_test(input_df, seq_len=10, max_players=22):
    possible_cols = ["x","y","speed","acceleration","vx","vy","dir_rad"]
    feat_cols = [c for c in possible_cols if c in input_df.columns]
    num_feats = len(feat_cols)
    static_cols = [c for c in ["height_inches", "player_weight", "player_position_encoded"] if c in input_df.columns]
    num_static_feats = len(static_cols)
    
    unique_plays = input_df[["game_id", "play_id"]].drop_duplicates().to_pandas()
    
    X_seq_list = []
    X_static_list = []
    X_initials_list = []
    meta_list = []
    
    for row in tqdm(unique_plays.itertuples(index=False), total=len(unique_plays), desc="Test Dataset"):
        gid, pid = row.game_id, row.play_id
        df_play = input_df[(input_df.game_id == gid) & (input_df.play_id == pid)].sort_values(["nfl_id", "frame_id"])
        players = df_play["nfl_id"].unique().to_cupy()
        
        X_seq_play = cp.zeros((max_players, seq_len, num_feats), dtype=cp.float32)
        X_static_play = cp.zeros((max_players, num_static_feats), dtype=cp.float32)
        initial_pos_play = np.zeros((max_players, 2), dtype=np.float32)
        
        players_meta = []
        
        for i, nfl_id_gpu in enumerate(players[:max_players]):
            nfl_id = nfl_id_gpu.item()
            players_meta.append(nfl_id)
            
            p_data = df_play[df_play.nfl_id == nfl_id]
            seq_vals = p_data[feat_cols].tail(seq_len).to_cupy()
            
            if len(seq_vals) > 0:
                X_seq_play[i, -len(seq_vals):, :] = seq_vals
                if len(p_data) > 0:
                    X_static_play[i, :] = p_data[static_cols].iloc[0].to_cupy()
                
                # --- FIX: Convertir a .item() (CPU) ---
                curr_x = seq_vals[-1, 0].item()
                curr_y = seq_vals[-1, 1].item()
                initial_pos_play[i, :] = [curr_x, curr_y]
                # --------------------------------------
                
        X_seq_list.append(cp.asnumpy(X_seq_play))
        X_static_list.append(cp.asnumpy(X_static_play))
        X_initials_list.append(initial_pos_play)
        meta_list.append({"game_id": gid, "play_id": pid, "players": players_meta})
        
    return {
        "X_seq": np.array(X_seq_list),
        "X_static": np.array(X_static_list),
        "X_initials": np.array(X_initials_list),
        "meta_list": meta_list
    }

print("✅ Funciones de Test REPARADAS.")

✅ Funciones de Test REPARADAS.


In [21]:
# =========================================================================
# BLOQUE FINAL: GENERACIÓN DE ARCHIVO DE SUMISIÓN
# =========================================================================

print("\n🚀 INICIANDO PROCESO DE SUMISIÓN...")

# 1. Cargar
print("1. Cargando datos...")
test_input, test_targets = load_test_data()

# 2. Features
print("2. Ingeniería de Features...")
fe_test = engineer_frame_features_gpu(test_input, test_input)
df_final_test = fe_test["df_final"]
df_final_test = reduce_memory_footprint_gpu(df_final_test)

# 3. Dataset
print("3. Construyendo tensores...")
test_dataset = build_play_level_dataset_test(
    input_df=df_final_test,
    seq_len=CONFIG["sequence_length"],
    max_players=CONFIG["max_players"]
)

X_seq_test = test_dataset["X_seq"]
X_static_test = test_dataset["X_static"]
X_initials_test = test_dataset["X_initials"]
meta_test = test_dataset["meta_list"]

# Transponer para que coincida con el modelo (N, 10, 22, F)
print(f"   Forma original: {X_seq_test.shape}")
X_seq_test = X_seq_test.transpose(0, 2, 1, 3)
print(f"   Forma corregida: {X_seq_test.shape}")

# 4. Predicción
print("4. Prediciendo...")
deltas_pred = model.predict(
    {"seq_input": X_seq_test, "static_input": X_static_test},
    batch_size=CONFIG["batch_size"],
    verbose=1
)

# 5. Reconstrucción
print("5. Reconstruyendo absolutos...")
initials_expanded = X_initials_test[:, :, np.newaxis, :]
abs_preds = initials_expanded + np.cumsum(deltas_pred, axis=2)

# 6. CSV
print("6. Guardando CSV...")
submission_rows = []

for i, meta in enumerate(meta_test):
    gid, pid = meta["game_id"], meta["play_id"]
    players = meta["players"]
    
    for p_idx, nfl_id in enumerate(players):
        for frame in range(CONFIG["max_frames_to_predict"]):
            submission_rows.append({
                "game_play_id": f"{gid}_{pid}",
                "nfl_id": int(nfl_id),
                "step": frame + 1,
                "x": float(abs_preds[i, p_idx, frame, 0]),
                "y": float(abs_preds[i, p_idx, frame, 1])
            })

sub_df = pd.DataFrame(submission_rows)
sub_df = sub_df[["game_play_id", "nfl_id", "step", "x", "y"]]
sub_df.to_csv("submission.csv", index=False)

print("\n" + "="*50)
print(f"✅ ¡ARCHIVO GENERADO! Filas: {len(sub_df)}")
print("="*50)


🚀 INICIANDO PROCESO DE SUMISIÓN...
1. Cargando datos...
📂 Cargando archivos CSV de Test...
2. Ingeniería de Features...
3. Construyendo tensores...


Test Dataset:   0%|          | 0/143 [00:00<?, ?it/s]

   Forma original: (143, 22, 10, 7)
   Forma corregida: (143, 10, 22, 7)
4. Prediciendo...
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step
5. Reconstruyendo absolutos...
6. Guardando CSV...

✅ ¡ARCHIVO GENERADO! Filas: 17580


In [22]:
import os
import shutil
import pandas as pd

print("🔍 Verificando archivo de sumisión...")

# 1. Verificar si existe en el directorio actual
if os.path.exists("submission.csv"):
    print("✅ ¡Encontrado! El archivo 'submission.csv' existe en el directorio actual.")
    
    # 2. Verificar contenido rápido
    df_check = pd.read_csv("submission.csv")
    print(f"   - Filas encontradas: {len(df_check)}")
    print(f"   - Columnas: {list(df_check.columns)}")
    
    # 3. Mover a /kaggle/working por seguridad (si no está ahí)
    # En Kaggle, el directorio de salida por defecto es /kaggle/working
    current_dir = os.getcwd()
    target_dir = "/kaggle/working"
    
    if current_dir != target_dir:
        print(f"⚠️ El archivo está en {current_dir}, moviéndolo a {target_dir}...")
        try:
            shutil.copy("submission.csv", "/kaggle/working/submission.csv")
            print("✅ Archivo copiado exitosamente a la carpeta de salida.")
        except Exception as e:
            print(f"❌ Error al mover: {e}")
    else:
        print("✅ El archivo ya está en la carpeta correcta (/kaggle/working).")

else:
    print("❌ ERROR CRÍTICO: No se encuentra 'submission.csv'.")
    print("   -> ¿Ejecutaste el bloque anterior ('BLOQUE FINAL')?")
    print("   -> Revisa si hubo algún error rojo en ese bloque.")

print("\n📂 Listando archivos en el directorio de salida:")
print(os.listdir("/kaggle/working"))

🔍 Verificando archivo de sumisión...
✅ ¡Encontrado! El archivo 'submission.csv' existe en el directorio actual.
   - Filas encontradas: 17580
   - Columnas: ['game_play_id', 'nfl_id', 'step', 'x', 'y']
✅ El archivo ya está en la carpeta correcta (/kaggle/working).

📂 Listando archivos en el directorio de salida:
['best_football_model.keras', 'submission.csv', '.virtual_documents', 'training_history.pkl', 'data_play_level.npz', 'meta_list.pkl']
